In [1]:
import os
from xopt import Evaluator, VOCS, Xopt
from xopt.generators import get_generator
from Genesis_functions import *
from TaperFunctions import *
Genesis4.apply_taper = apply_taper
import pandas as pd
import yaml
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from lume_model.variables import ScalarInputVariable, ScalarOutputVariable
from lume_model.models import TorchModel
from lume_model.utils import variables_as_yaml
import shutil



import os
from xopt import Evaluator
from xopt import VOCS
import math
from xopt.generators import get_generator 
from xopt import Xopt

#from xopt import AsynchronousXopt as Xopt
from Genesis_functions import *

from TaperFunctions import *

Genesis4.apply_taper = apply_taper
import pandas as pd
import yaml


import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import joblib
import matplotlib.pyplot as plt

import torch
import joblib
import pandas as pd
from botorch.models.transforms.input import AffineInputTransform

from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import random

from collections import deque

  


# Function to set seed
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set the seed for reproducibility
set_seed(42)


def limit_points_per_range(dataset, step=100, max_points=1000):
    """
    Reduces the number of points in each energy range (in steps of `step`) 
    to be less than or equal to `max_points`.

    Args:
    - dataset (pd.DataFrame): The dataset containing an 'energy' column.
    - step (int): Step size for dividing the energy range (default: 100).
    - max_points (int): Maximum number of points allowed in each range (default: 1000).

    Returns:
    - pd.DataFrame: The filtered dataset with reduced points per range.
    """
    # Define bins for the energy ranges
    bins = range(int(dataset['energy'].min()), int(dataset['energy'].max()) + step, step)
    
    # Add a new column for the bin each energy belongs to
    dataset['energy_bin'] = pd.cut(dataset['energy'], bins=bins, right=False)
    
    # Limit the number of points per bin
    filtered_dataset = dataset.groupby('energy_bin').apply(
        lambda x: x.sample(n=min(len(x), max_points), random_state=42)
    ).reset_index(drop=True)
    
    # Drop the temporary energy_bin column
    filtered_dataset = filtered_dataset.drop(columns=['energy_bin'])
    
    return filtered_dataset


def scale_and_save_transform(features, dump_filename, dataset):
    """
    Scales the provided features in the dataset using AffineInputTransform, saves the transform, 
    and returns the scaled dataset.
    
    Args:
    - features (list of str): List of feature names to scale.
    - dump_filename (str): The filename to save the AffineInputTransform object.
    - dataset (pd.DataFrame): The dataset containing the features to scale.

    Returns:
    - pd.DataFrame: The dataset with scaled features.
    """
    normalizations = {}

    # Collect min and max values for each feature
    for feature in features:
        min_val = dataset[feature].min()
        max_val = dataset[feature].max()
        scale =  (max_val - min_val) if max_val != min_val else 1.0  # Avoid division by zero
        shift = min_val 
        normalizations[f"{feature}_scale"] = scale
        normalizations[f"{feature}_mean"] = shift

    # Combine scaling factors and offsets into tensors
    coefficient = torch.tensor([normalizations[f"{feature}_scale"] for feature in features])
    offset = torch.tensor([normalizations[f"{feature}_mean"] for feature in features])


    print(coefficient, offset)
    # Create a single AffineInputTransform for all features
    affine_transform = AffineInputTransform(
        len(features), 
        coefficient=coefficient, 
        offset=offset
    )

    # Apply the transform to the dataset
    features_data = torch.tensor(dataset[features].values, dtype=torch.float32)  # Convert features to a tensor
    scaled_features = affine_transform.transform(features_data)

    #scaled_features = affine_transform.forward(features_data)

    # Add the scaled features back to the dataset
    for i, feature in enumerate(features):
        dataset[f'scaled_{feature}'] = scaled_features[:, i].numpy()

    # Save the transform for later use
  #  joblib.dump(affine_transform, dump_filename)
    torch.save(affine_transform, dump_filename)

    return dataset

def load_transform_and_apply(dump_filename, features, dataset):
    """
    Loads the AffineInputTransform from the specified file and uses it to apply the transform (scaling)
    to the provided dataset.
    
    Args:
    - dump_filename (str): The filename where the AffineInputTransform object is saved.
    - features (list of str): List of feature names to transform.
    - dataset (pd.DataFrame): The dataset containing the original features to be transformed.

    Returns:
    - pd.DataFrame: The dataset with scaled features.
    """
    # Load the saved AffineInputTransform
    affine_transform = torch.load(dump_filename)
    
    # Extract the features from the dataset
    feature_data = torch.tensor(dataset[features].values)  # Convert to tensor
    
    # Apply the forward transform (scaling)
    scaled_features = affine_transform.transform(feature_data)

    # Add the scaled features back to the dataset
    for i, feature in enumerate(features):
        dataset[f'scaled_{feature}'] = scaled_features[:, i].numpy()

    return dataset

def apply_inverse_transform(dump_filename, features, dataset):
    """
    Loads the AffineInputTransform from the specified file and uses it to apply the inverse transform (rescaling)
    to the provided dataset.
    
    Args:
    - dump_filename (str): The filename where the AffineInputTransform object is saved.
    - features (list of str): List of feature names to rescale (apply inverse transform).
    - dataset (pd.DataFrame or np.ndarray): The dataset containing the scaled features to rescale.

    Returns:
    - np.ndarray: The rescaled features in their original scale.
    """
    # Load the saved AffineInputTransform
    affine_transform = torch.load(dump_filename)

    # Convert dataset to tensor if necessary
    if isinstance(dataset, pd.DataFrame):
        scaled_features_data = torch.tensor(dataset[[f'scaled_{feature}' for feature in features]].values)  # Convert to tensor
    else:
        scaled_features_data = torch.tensor(dataset)  # If already a NumPy array or tensor

    # Apply the inverse transform to rescale the data
    rescaled_features = affine_transform.untransform(scaled_features_data)

    return rescaled_features.numpy()  # Return as NumPy array


# In[28]:


from lume_model.models import TorchModel, TorchModule
from lume_model.variables import ScalarInputVariable, ScalarOutputVariable






# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open('dump_cnsga.yml', 'r') as file:
    data_load = yaml.safe_load(file)

data = pd.DataFrame(data_load['data'])
filtered_data = data[data['xopt_error'] == False]

alphax_values = filtered_data.alphax
alphay_values = filtered_data.alphay
betax_values = filtered_data.betax
betay_values = filtered_data.betay
emitx_values = filtered_data.emitx
emity_values = filtered_data.emity
taper_values = filtered_data.taper
energy_values = filtered_data.energy
bandwidth_values = filtered_data.bandwidth_FWHM
peak_intensity=filtered_data.peak_intensity


X_test = Xopt.from_file("dump_testsdata.yml")
X_test_filter = X_test.data[X_test.data['xopt_error'] == False]

alphax_test = X_test_filter.alphax
alphay_test = X_test_filter.alphay
betax_test = X_test_filter.betax
betay_test = X_test_filter.betay
taper_test = X_test_filter.taper
emitx_test = X_test_filter.emitx
emity_test = X_test_filter.emity
energy_test = X_test_filter.energy
bandwidth_test = X_test_filter.bandwidth_FWHM
peak_intensity_test=X_test_filter.peak_intensity
# Save test data to GPU tensors
inputs_to_nn = torch.tensor([alphax_test, alphay_test, betax_test, betay_test, emitx_test, emity_test, taper_test], dtype=torch.float32).T.to(device)
torch.save(inputs_to_nn, 'lume_model/info/test_model_inputs.pt')

outputs_to_nn = torch.tensor([energy_test, bandwidth_test,peak_intensity_test], dtype=torch.float32).T.to(device)
torch.save(outputs_to_nn, 'lume_model/info/test_model_outputs.pt')

vocs = X_test.vocs
input_variables = [ScalarInputVariable(name=key, default=(value[0] + value[1]) / 2, value_range=value) for key, value in vocs.variables.items()]
output_variables = [ScalarOutputVariable(name=observable) for observable in vocs.observables]
variables_as_yaml(input_variables, output_variables, file='./lume_model/sim_variables.yml')


dataset =pd.DataFrame({
        'alphax': alphax_values,
        'alphay': alphay_values,
        'betax': betax_values,
        'betay': betay_values,
        'emitx': emitx_values,
        'emity': emity_values,
        'taper': taper_values,
        'energy': energy_values,
        'bandwidth': abs(bandwidth_values),
        'peak_intensity':peak_intensity})

duplicate_columns = ['alphax', 'alphay', 'betax', 'betay', 'taper', 'emitx', 'emity']



with open('../../generatedata_energy/dump_cnsga.yml', 'r') as file:
    data_load2 = yaml.safe_load(file)

data2 = pd.DataFrame(data_load2['data'])
filtered_data2 = data2[data2['xopt_error'] == False]

alphax_values2 = filtered_data2.alphax
alphay_values2 = filtered_data2.alphay
betax_values2 = filtered_data2.betax
betay_values2 = filtered_data2.betay
emitx_values2 = filtered_data2.emitx
emity_values2 = filtered_data2.emity
taper_values2 = filtered_data2.taper
energy_values2 = filtered_data2.energy
bandwidth_values2 = filtered_data2.bandwidth_FWHM
peak_intensity2=filtered_data2.peak_intensity

dataset2 = pd.DataFrame({
        'alphax': alphax_values2,
        'alphay': alphay_values2,
        'betax': betax_values2,
        'betay': betay_values2,
        'emitx': emitx_values2,
        'emity': emity_values2,
        'taper': taper_values2,
        'energy': energy_values2,
        'bandwidth': abs(bandwidth_values2),
        'peak_intensity':peak_intensity})

combined_dataset = pd.concat([dataset, dataset2], ignore_index=True)
combined_dataset = combined_dataset.drop_duplicates(subset=duplicate_columns, keep='first')  

####combined_dataset=dataset

combined_dataset = combined_dataset[combined_dataset['bandwidth'] <=150]



test_data_sample = combined_dataset.sample(n=100, random_state=42)

# Extract features and target for test dat
features_columns = ['alphax', 'alphay', 'betax', 'betay', 'emitx', 'emity', 'taper']
target_column = ['energy', 'bandwidth', 'peak_intensity']

test_features = test_data_sample[features_columns].values
test_target = test_data_sample[target_column].values

# Convert to tensors
inputs_to_nn2 = torch.tensor(test_features, dtype=torch.float32).to(device)
outputs_to_nn2 = torch.tensor(test_target, dtype=torch.float32).to(device)

# Save the tensors
torch.save(inputs_to_nn2, 'lume_model/info/test_model_inputs2.pt')
torch.save(outputs_to_nn2, 'lume_model/info/test_model_outputs2.pt')


print('combine', len(combined_dataset), 'data1', len(data), 'data2,', len(data2))


# Remove the selected data points from the combined dataset
combined_dataset = combined_dataset.drop(test_data_sample.index)



print("Test dataset saved and removed from the combined dataset successfully!")



print(len(combined_dataset), 'before')

#combined_dataset = combined_dataset[combined_dataset['energy'] >= 200]
#combined_dataset = combined_dataset[combined_dataset['bandwidth'] <=200]
#combined_dataset = limit_points_per_range(combined_dataset, step=50, max_points=400)
#print("Test dataset saved and removed from the combined dataset successfully!")


print('combined_data', len(combined_dataset))

ModuleNotFoundError: No module named 'Genesis_functions'